In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import random

In [ ]:
def get_random_dates(start_time, n_cases, epidemic_duration):
    """Generate random onset dates for cases.
    """
    date_ints = random.sample(range(epidemic_duration), n_cases)
    return [start_time + timedelta(days=i_time) for i_time in date_ints]

def get_dates_df_from_list(dates_list):
    """Dataframe with every subsequent case infected by the first one.
    """
    case_data_df = pd.DataFrame(sorted(dates_list), columns=["date"])
    case_data_df["infector"] = 0
    return case_data_df

def generate_random_infectors(cases_df):
    """Randomly assign each case a preceding infector.
    """
    for i_case in cases_df.index[1:]:
        cases_df.loc[i_case, "infector"] = random.randint(0, i_case - 1)

In [ ]:
case_dates = get_random_dates(datetime(2020, 1, 1), 20, 100)
case_data_df = get_dates_df_from_list(case_dates)
generate_random_infectors(case_data_df)
case_data_df

In [ ]:
case_data_df["generation_time"] = case_data_df.index - case_data_df["infector"]

In [ ]:
import networkx as nx
G = nx.DiGraph()
for index, row in case_data_df.iterrows():
    G.add_node(row["infector"])  # Add preceding case as node
    G.add_node(index)
    G.add_edge(row["infector"], index, length=row["generation_time"]) 

pos = {}
for index, node in enumerate(G.nodes()):
    pos[node] = (case_data_df.loc[index, "date"], node)
nx.draw(G, pos, node_size=40)